In [1]:
import pandas as pd

urls_train = pd.read_parquet('Training.parquet')
urls_test = pd.read_parquet('Testing.parquet')

urls = pd.concat([urls_train, urls_test]).reset_index().drop(['index'], axis = 1)
urls['status'] = urls.status.replace(['legitimate', 'phishing'], [0, 1])

/var/folders/7y/z4ckz2m16zvb_lc_v2ddsc000000gn/T/ipykernel_3632/4080705752.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  urls['status'] = urls.status.replace(['legitimate', 'phishing'], [0, 1])


In [2]:
urls_temp = urls.drop('url', axis=1, inplace=False)

In [3]:
# kolumny z wysoka korelacja ze statusem
cols = urls_temp.corr()[abs(urls_temp.corr()['status']) > 0.2]['status'].index
cols

Index(['length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_qm', 'nb_eq',
       'nb_slash', 'nb_www', 'ratio_digits_url', 'ratio_digits_host',
       'tld_in_subdomain', 'prefix_suffix', 'shortest_word_host',
       'longest_words_raw', 'longest_word_path', 'phish_hints',
       'nb_hyperlinks', 'ratio_intHyperlinks', 'empty_title',
       'domain_in_title', 'domain_age', 'google_index', 'page_rank', 'status'],
      dtype='object')

In [4]:
# usuniecie trudnych do wyciagniecia danych
cols = cols.difference(['ip', 'prefix_suffix', 'phish_hints', 'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_intHyperlinks', 'domain_in_title', 'domain_age', 'google_index', 'page_rank'])
urls = urls[cols]

cols_cat = [col for col in urls.columns if len(urls[col].value_counts()) == 2 and 'nb' not in col]
# te kolumny numeryczne potrzebne do normalizacji
cols_num = list(set(urls.columns) - set(cols_cat) - {'url'})

In [5]:
# normalizacja
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))

for col in cols_num:
    urls[col] = scaler.fit_transform(pd.DataFrame(urls[col]))


In [6]:
# podzial na zmienne objasniajace i objasniane
cols_X = list(urls.columns)
cols_X.remove('status')

urls_X = urls[cols_X]
urls_y = urls['status']

In [7]:
# pca
from sklearn.decomposition import PCA

pca = PCA()

pca.fit(urls_X)

sum = 0
global pca_dim
pca_dim = 0
while (sum < 0.95):
    sum += pca.explained_variance_ratio_[pca_dim]
    pca_dim += 1

pca = PCA(n_components=pca_dim)

urls_X = pd.DataFrame(pca.fit_transform(urls_X))

7

In [22]:
# zapisanie danych
urls = pd.concat((urls_X, urls_y), axis=1)
urls.to_csv("data_clean.csv", index=False)